In [1]:
# review crawler

import pandas as pd
import time
import urllib.request
from lxml.html import fromstring
from bs4 import BeautifulSoup
import re
import requests
import time
import csv
from selenium import webdriver
from requests.exceptions import RequestException

# download html
def download(url):
    print('Downloading:', url)
    request = urllib.request.Request(url)
    request.add_header('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36')
    resp = urllib.request.urlopen(request)
    html = resp.read().decode('utf-8')
    return html


def generate_movie_review_list_link():
    start_url = download('https://www.imdb.com/chart/top/?ref_=nv_mv_250')
    domain = 'https://www.imdb.com/'
    start_soup = BeautifulSoup(start_url)
    
    movie_reviews_url_list = []
    
    for k in range(250):
        sub_html = start_soup.find_all('tbody')[0].find_all('a')[2*k+1].get('href')
        url = domain + sub_html
        # print("url: ", url)
        id_pattern = re.compile(r'(?<=tt)\d+(?=/?)')
        movie_id = str(id_pattern.search(url).group()).zfill(7)    #imdb电影id
        print("movie_id: ", movie_id)
        movie_reviews_url = "https://www.imdb.com/title/tt"+str(movie_id)+"/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0"
        # print("movie_reviews_url: ", movie_reviews_url)
        movie_reviews_url_list.append([movie_id,movie_reviews_url])

    return movie_reviews_url_list

#获取imdb电影评论页面所有非spoiler的用户评论
def get_imdb_movie_review(url,movieId):
    NETWORK_STATUS = True  # 判断状态变量
    URL = url
    print(URL)
    try:
        response = requests.get(URL)
        if response.status_code == 200:
            original_html = response.text
            original_soup = BeautifulSoup(original_html,'lxml')
            #需要下载Chromedriver
            driver = webdriver.Chrome()
            driver.get(URL)

            # 判断是否需要模拟点击load more 按钮
            i=0
            while(i<5):
                i+=1
                temp_response = driver.page_source
                temp_html = temp_response
                temp_soup = BeautifulSoup(temp_html,'lxml')
                load_more = temp_soup.select('.ipl-load-more__button')
                #需要模拟点击
                if(len(load_more) == 1):
                    button_load_more = driver.find_element_by_class_name('ipl-load-more__button')
                    button_is_or_not_visible = driver.find_element_by_class_name('ipl-load-more__button').is_displayed()
                    if(button_is_or_not_visible is True and i<5):       #button按钮不可见时停止点击
                        button_load_more.click()
                        time.sleep(3)
                    else:
                        break
                else:
                    break

            final_response = driver.page_source
            html = final_response
            soup = BeautifulSoup(html,'lxml')

            movie_id = movieId
            reviews = soup.select('.review-container')
            movie_reviews_list = [[0] * 4 for j in range(200)]

            i = 0
            for review in reviews:
                header = review.select_one('.display-name-date')
                user_link = header.select_one('a')['href']
                user_id_pattern = re.compile(r'(?<=ur)\d+(?=/?)')
                user_id = int(user_id_pattern.search(user_link).group())    #用户id
                review_date = header.select_one('.review-date').string      #用户评论时间

                content = review.select_one('.content')
                user_review = content.select('.text.show-more__control')     #用户评论
                movie_reviews_list[i][0] = movie_id
                movie_reviews_list[i][1] = review_date
                movie_reviews_list[i][2] = user_id
                movie_reviews_list[i][3] = user_review

                i += 1
                #print(i,user_id,review_date,user_review,movie_id)
                time.sleep(0.01)
                if i>= 100:
                    break

            driver.close()      #关闭网页
            return movie_reviews_list
        else:
            print("Error when request URL")
    except RequestException:
        print("Request Failed")
        return None
    except requests.exceptions.Timeout:
        NETWORK_STATUS = False  # 请求超时改变状态
        if NETWORK_STATUS == False:
            #'''请求超时'''
            driver.close()  # 关闭网页
            print('请求超时，重复请求')
            get_imdb_movie_review(URL, movieId)

if __name__ == '__main__':
    count = 0
    with open('imdb_movie_review_info.csv', 'w', newline="",encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, dialect=("excel"))
        csvwriter.writerow(["movieId", "reviewDate", "userId", "userReview"])
        movie_review_url_list =  generate_movie_review_list_link()
        print("movie_review_url_list: ", movie_review_url_list)
        j = 0

        while (j < 250):
            print("第" + str(j + 1) + "部电影的评论")
            # 只收集前100条 order by helpfulness
            l = [[0] * 4 for j in range(200)]
            l = get_imdb_movie_review(movie_review_url_list[j][1], movie_review_url_list[j][0])
            k = 0
            if(l is None):
                j += 1
                continue

            while(l[k][2] != 0):
                csvwriter.writerow(l[k])
                k += 1
                count += 1
            j += 1

    print("爬虫完毕，共爬取"+str(count)+"个用户评论")

Downloading: https://www.imdb.com/chart/top/?ref_=nv_mv_250
movie_id:  0111161
movie_id:  0068646
movie_id:  0071562
movie_id:  0468569
movie_id:  0050083
movie_id:  0108052
movie_id:  0167260
movie_id:  0110912
movie_id:  0060196
movie_id:  0120737
movie_id:  0137523
movie_id:  0109830
movie_id:  1375666
movie_id:  0167261
movie_id:  0080684
movie_id:  0133093
movie_id:  0099685
movie_id:  0073486
movie_id:  0047478
movie_id:  0114369
movie_id:  0118799
movie_id:  0317248
movie_id:  0102926
movie_id:  0038650
movie_id:  0076759
movie_id:  0120815
movie_id:  0120689
movie_id:  0245429
movie_id:  0816692
movie_id:  6751668
movie_id:  0110413
movie_id:  0056058
movie_id:  0114814
movie_id:  0110357
movie_id:  0253474
movie_id:  0088763
movie_id:  0103064
movie_id:  0120586
movie_id:  0027977
movie_id:  0172495
movie_id:  0054215
movie_id:  0407887
movie_id:  0021749
movie_id:  1675434
movie_id:  2582802
movie_id:  0095327
movie_id:  0482571
movie_id:  0064116
movie_id:  0034583
movie_id: